## 시설물 보수보강이력 목록

In [1]:
from common import commonFunc as cf
import pandas as pd
pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "시설물 보수보강이력 목록"
APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/시설물 보수보강이력 목록/selectIoFmMnrHstList.csv'

In [2]:
# 개별 시설물 보수보강이력 목록 수
g_totalCount = 0

In [3]:
##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
SERVICENAME

'selectIoFmMnrHstList'

In [4]:
# REQPARAM = targetData["요청변수"].values[0]
# REQPARAM = REQPARAM.split(",")
# PRIMARYKEY = targetData["기본키"].values[0]
# REQPARAM

In [5]:
JSONKEY = "detail1"
DUMMY = 0

In [6]:
#임시 
imsiDf = pd.read_csv(\
         "../output/건설사업정보시스템/시설물 목록/selectIoFmMngList.csv", encoding="ms949")

In [7]:
dfCombi = imsiDf.loc[:,["fcno"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)
dfCombiLen = dfCombi.shape[0]
dfCombiLen

7913

## 시설물 보수보강이력 목록 전용 스크랩 함수

In [8]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal"):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
            response = requests.get(inUrl+queryParams,verify=False)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata != "NORMAL_SERVICE"):
                        print("SERVER ERROR ",jsondata)
                        break
                    
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 items 정보에 세부정보가 더 있음
               
                jsondata = response.json()["response"]["body"][jsonkey]
                if( jsondata == []):
                    print("{} page is empty".format(i))
                    break
                if( jsonkey == "detail1"):
                    jsondata["index"]=[0]
                rowData = pd.DataFrame(jsondata)
                emptyPd = emptyPd.append(rowData)
                # fcno를 키로 기본정보와 보수보강이력 정보를 merge하기 위함
                print("fcno ",emptyPd['fcno'].values[0])
                fcno = emptyPd['fcno'].values[0]
                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    global g_totalCount
                    if g_totalCount == 0:
                        g_totalCount = int(response.json()["response"]["body"]["totalCount"])
                    jsondata = response.json()["response"]["body"]["items"]
                    if( jsondata == []):
                        print("items is empty")
                    else:
#                         jsondata["index"]=[0]
                        rowData = pd.DataFrame(jsondata)
                        
                        rowData['fcno'] = fcno # 시설물 정보에 fcno칼럼을 수동으로 추가 (머지키로 활용하기 위함)
                        #emptyPd = pd.concat([emptyPd,rowData],axis=1).reindex(emptyPd.index)
                        emptyPd = pd.merge(left = emptyPd , right = rowData, on = "fcno")
                    
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

In [14]:

numOfRows = 100
PAGEYN=0

In [15]:
resultDfMerged = pd.DataFrame()
resultDf = pd.DataFrame()

for i in range(0,dfCombiLen):
    fcnoValue = dfCombi.loc[i].fcno
    pageNo = 1
    global g_totalCount
    g_totalCount = 0
    print("i=",i," fcnoValue= ",fcnoValue)
    while True:
        BASEPARAM={"serviceKey":APIKEY,"pageNo": pageNo, 'fcno':fcnoValue, 'numOfRows':numOfRows, "type":"json"}
        resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
        if resultDf.empty: # 정상 데이터가 없는 경우
            break
        else:
            resultDfMerged = resultDfMerged.append(resultDf) 
            g_totalCount -= numOfRows
            print("g_totalCount : ", g_totalCount, " numOfRows : ", numOfRows)
            if(g_totalCount <= 0):
                break
            pageNo += 1
        

        

i= 0  fcnoValue=  017F2520A338CB0F98899109C3F1E127
1 page scraping start
fcno  017F2520A338CB0F98899109C3F1E127
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 1  fcnoValue=  017F25212C92C63041C749D8850C27F0
1 page scraping start
fcno  017F25212C92C63041C749D8850C27F0
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 2  fcnoValue=  017F2521A0A3E80974D725F1B6DB68EF
1 page scraping start
fcno  017F2521A0A3E80974D725F1B6DB68EF
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 3  fcnoValue=  017F25224310A850FB962985F3EC1D60
1 page scraping start
fcno  017F25224310A850FB962985F3EC1D60
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 4  fcnoValue=  017F2524A82A3053A24976923A690EA9
1 page scraping start
fcno  017F2524A82A3053A24976923A690EA9
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 5  fcnoValue=  017F252518748A0FACF1C149F048DDE7
1 page scraping start
fcno  017F252518748A0FACF1C149F048DDE7
>to

fcno  017B5BE646FCD2F75FCD5ED0AE53AB04
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 47  fcnoValue=  017B5BE6B96501F8236A3BC70C8FFAA6
1 page scraping start
fcno  017B5BE6B96501F8236A3BC70C8FFAA6
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 48  fcnoValue=  017B5BE70E0BD65D01FEEE3327B1590F
1 page scraping start
fcno  017B5BE70E0BD65D01FEEE3327B1590F
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 49  fcnoValue=  017B5BE7709C23A7AD99F759F9BB5492
1 page scraping start
fcno  017B5BE7709C23A7AD99F759F9BB5492
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 50  fcnoValue=  017B5BE7C20285F9DF67F8429764001C
1 page scraping start
fcno  017B5BE7C20285F9DF67F8429764001C
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 51  fcnoValue=  017B5BEA02E68B4C0C12D73F54FC829C
1 page scraping start
fcno  017B5BEA02E68B4C0C12D73F54FC829C
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 52 

fcno  0177CD93229F60B61CE109D9AA46F983
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 94  fcnoValue=  017D7474887EABDE8672959B42FAA6E2
1 page scraping start
fcno  017D7474887EABDE8672959B42FAA6E2
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 95  fcnoValue=  017D74748469C2A89CAA9D29D4F890B3
1 page scraping start
fcno  017D74748469C2A89CAA9D29D4F890B3
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 96  fcnoValue=  017D747489348DCDA8C446EA73892749
1 page scraping start
fcno  017D747489348DCDA8C446EA73892749
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 97  fcnoValue=  017D7474882E65DCC52BF9AD81528B2A
1 page scraping start
fcno  017D7474882E65DCC52BF9AD81528B2A
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 98  fcnoValue=  017D747488DB9092C036ADD8359CF0D0
1 page scraping start
fcno  017D747488DB9092C036ADD8359CF0D0
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 99 

fcno  0176B23F07FA5D7D007CB83B6DED5007
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 140  fcnoValue=  0176B26462F8476277D1101C61534381
1 page scraping start
fcno  0176B26462F8476277D1101C61534381
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 141  fcnoValue=  0176B23F07A09C2D4C9702457FF36CAE
1 page scraping start
fcno  0176B23F07A09C2D4C9702457FF36CAE
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 142  fcnoValue=  0176B26464C54303CD83EC0448856D0F
1 page scraping start
fcno  0176B26464C54303CD83EC0448856D0F
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 143  fcnoValue=  0176B23F06DCCF480D66094103909D2C
1 page scraping start
fcno  0176B23F06DCCF480D66094103909D2C
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 144  fcnoValue=  0176B23F04E212B02CAD98F5C6E462A9
1 page scraping start
fcno  0176B23F04E212B02CAD98F5C6E462A9
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i

fcno  01701DF6E44FE8E5FB62DAB3FCF9C14A
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 186  fcnoValue=  01700DA666D7D524DF209BB80427C7D5
1 page scraping start
fcno  01700DA666D7D524DF209BB80427C7D5
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 187  fcnoValue=  01700A1FE976402ACDE96EC21005CF11
1 page scraping start
fcno  01700A1FE976402ACDE96EC21005CF11
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 188  fcnoValue=  017009E712159E6A8FF59653E82F5C80
1 page scraping start
fcno  017009E712159E6A8FF59653E82F5C80
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 189  fcnoValue=  017009DD795C803EA7680048410746BA
1 page scraping start
fcno  017009DD795C803EA7680048410746BA
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 190  fcnoValue=  017009B46AA9C115DB1053189E12DB11
1 page scraping start
fcno  017009B46AA9C115DB1053189E12DB11
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i

...

fcno  E673B64265671C52E034080020A9316A
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 7695  fcnoValue=  EFE3B42F11060191E0340003BA2D2647
1 page scraping start
fcno  EFE3B42F11060191E0340003BA2D2647
>totalCount 6
0 no pageNo
g_totalCount :  -94  numOfRows :  100
i= 7696  fcnoValue=  E673B64270FD1C52E034080020A9316A
1 page scraping start
fcno  E673B64270FD1C52E034080020A9316A
>totalCount 3
0 no pageNo
g_totalCount :  -97  numOfRows :  100
i= 7697  fcnoValue=  E673B64269011C52E034080020A9316A
1 page scraping start
fcno  E673B64269011C52E034080020A9316A
>totalCount 4
0 no pageNo
g_totalCount :  -96  numOfRows :  100
i= 7698  fcnoValue=  E673B64265691C52E034080020A9316A
1 page scraping start
fcno  E673B64265691C52E034080020A9316A
>totalCount 11
0 no pageNo
g_totalCount :  -89  numOfRows :  100
i= 7699  fcnoValue=  E673B6426DD51C52E034080020A9316A
1 page scraping start
fcno  E673B6426DD51C52E034080020A9316A
>totalCount 1
0 no pageNo
g_totalCount :  -99  numOfRows :  100


fcno  E673B64262F11C52E034080020A9316A
>totalCount 10
0 no pageNo
g_totalCount :  -90  numOfRows :  100
i= 7741  fcnoValue=  E673B64265151C52E034080020A9316A
1 page scraping start
fcno  E673B64265151C52E034080020A9316A
>totalCount 8
0 no pageNo
g_totalCount :  -92  numOfRows :  100
i= 7742  fcnoValue=  E673B64265CA1C52E034080020A9316A
1 page scraping start
fcno  E673B64265CA1C52E034080020A9316A
>totalCount 1
0 no pageNo
g_totalCount :  -99  numOfRows :  100
i= 7743  fcnoValue=  EFE3B42F12460191E0340003BA2D2647
1 page scraping start
fcno  EFE3B42F12460191E0340003BA2D2647
>totalCount 16
0 no pageNo
g_totalCount :  -84  numOfRows :  100
i= 7744  fcnoValue=  EFE3B42F12490191E0340003BA2D2647
1 page scraping start
fcno  EFE3B42F12490191E0340003BA2D2647
>totalCount 21
0 no pageNo
g_totalCount :  -79  numOfRows :  100
i= 7745  fcnoValue=  E673B642681F1C52E034080020A9316A
1 page scraping start
fcno  E673B642681F1C52E034080020A9316A
>totalCount 5
0 no pageNo
g_totalCount :  -95  numOfRows :  100

fcno  67CBBE3FD5664B72B3B5C06F3D287C81
>totalCount 8
0 no pageNo
g_totalCount :  -92  numOfRows :  100
i= 7787  fcnoValue=  E673B6426CBF1C52E034080020A9316A
1 page scraping start
fcno  E673B6426CBF1C52E034080020A9316A
>totalCount 14
0 no pageNo
g_totalCount :  -86  numOfRows :  100
i= 7788  fcnoValue=  E673B642627F1C52E034080020A9316A
1 page scraping start
fcno  E673B642627F1C52E034080020A9316A
>totalCount 10
0 no pageNo
g_totalCount :  -90  numOfRows :  100
i= 7789  fcnoValue=  E673B64262691C52E034080020A9316A
1 page scraping start
fcno  E673B64262691C52E034080020A9316A
>totalCount 16
0 no pageNo
g_totalCount :  -84  numOfRows :  100
i= 7790  fcnoValue=  E673B64262701C52E034080020A9316A
1 page scraping start
fcno  E673B64262701C52E034080020A9316A
>totalCount 6
0 no pageNo
g_totalCount :  -94  numOfRows :  100
i= 7791  fcnoValue=  E673B64268F11C52E034080020A9316A
1 page scraping start
fcno  E673B64268F11C52E034080020A9316A
>totalCount 5
0 no pageNo
g_totalCount :  -95  numOfRows :  100

fcno  AB0637E0C9F2475386EC8763C19DC1EA
>totalCount 12
0 no pageNo
g_totalCount :  -88  numOfRows :  100
i= 7833  fcnoValue=  C1427B56A2B9482dA3C17645FC7E66ED
1 page scraping start
fcno  C1427B56A2B9482dA3C17645FC7E66ED
>totalCount 5
0 no pageNo
g_totalCount :  -95  numOfRows :  100
i= 7834  fcnoValue=  A3A01831E5156865E0440003BA816705
1 page scraping start
fcno  A3A01831E5156865E0440003BA816705
>totalCount 2
0 no pageNo
g_totalCount :  -98  numOfRows :  100
i= 7835  fcnoValue=  8A247CDAA09E5237E054B099289CEB28
1 page scraping start
fcno  8A247CDAA09E5237E054B099289CEB28
>totalCount 2
0 no pageNo
g_totalCount :  -98  numOfRows :  100
i= 7836  fcnoValue=  E673B64263C01C52E034080020A9316A
1 page scraping start
fcno  E673B64263C01C52E034080020A9316A
>totalCount 10
0 no pageNo
g_totalCount :  -90  numOfRows :  100
i= 7837  fcnoValue=  EFE3B42F10C80191E0340003BA2D2647
1 page scraping start
fcno  EFE3B42F10C80191E0340003BA2D2647
>totalCount 3
0 no pageNo
g_totalCount :  -97  numOfRows :  100


fcno  FB0A5869B45025FFE0340003BA90E0BD
>totalCount 12
0 no pageNo
g_totalCount :  -88  numOfRows :  100
i= 7879  fcnoValue=  E0CB9DBCC13E469d966FC921103A00AB
1 page scraping start
fcno  E0CB9DBCC13E469d966FC921103A00AB
>totalCount 1
0 no pageNo
g_totalCount :  -99  numOfRows :  100
i= 7880  fcnoValue=  96B761D888C3175BE054B099289CEB28
1 page scraping start
fcno  96B761D888C3175BE054B099289CEB28
>totalCount 0
0 no pageNo
g_totalCount :  -100  numOfRows :  100
i= 7881  fcnoValue=  FB0AF1609C042896E0340003BA90E0BD
1 page scraping start
fcno  FB0AF1609C042896E0340003BA90E0BD
>totalCount 5
0 no pageNo
g_totalCount :  -95  numOfRows :  100
i= 7882  fcnoValue=  FB0AEA0DB9212880E0340003BA90E0BD
1 page scraping start
fcno  FB0AEA0DB9212880E0340003BA90E0BD
>totalCount 6
0 no pageNo
g_totalCount :  -94  numOfRows :  100
i= 7883  fcnoValue=  FB0AC62B68F42837E0340003BA90E0BD
1 page scraping start
fcno  FB0AC62B68F42837E0340003BA90E0BD
>totalCount 3
0 no pageNo
g_totalCount :  -97  numOfRows :  100


In [16]:
resultDfMerged.shape

(24029, 32)

In [17]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME)

시설물 보수보강이력 목록 save compled


In [18]:
resultDfMerged.shape

(24029, 32)